In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import copy
import warnings
warnings.filterwarnings('ignore')

import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig

from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader

from src.data import load_Ford_A, transform_data, MyDataset
from src.models import LSTM_net

from src.attacks import attack_procedure
from src.utils import save_experiment, load_disc_model
from src.config import get_attack, load_disc_config
from src.data import load_UCR


In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf
initialize(config_path='../config', version_base=None)
cfg = compose(config_name='attack_run_config.yaml')
print(OmegaConf.to_yaml(cfg))

dataset: PowerCons
slice: true
use_disc_check: false
test_run: false
model_folder: checkpoints/${dataset}/TS2Vec/
disc_path: results/${dataset}/IFGSM
save_path: ${disc_path}/${attack_type}
model_id_attack: 0
attack_model:
  _target_: src.models.TS2VecClassifier
  dropout: None
disc_model:
  _target_: src.models.TS2VecClassifier
  dropout: 0.0
  dropout_ts2vec: 0.0
  device: 2
  n_layers: 2
disc_model_check:
  _target_: src.models.TS2VecClassifier
  dropout: 0.0
  dropout_ts2vec: 0.0
  device: 2
  n_layers: 2
cuda: cuda:1
attack_type: fgsm_attack
n_iterations: 50
all_eps:
- 0.03
- 0.07
- 0.1
- 0.13
- 0.17
train_mode: false
batch_size: 64
alphas:
- 10
- 1
- 0.001
- 0.01
- 0.1
disc_check_params:
  model_id: 0
  model_name: fgsm_attack/fgsm_attack_eps=0.1_nsteps=4
list_reg_model_params:
- model_id: 0
  model_name: fgsm_reg_attack/fgsm_reg_attack_eps=0.1_alpha=0.01_nsteps=4
- model_id: 0
  model_name: fgsm_reg_attack/fgsm_reg_attack_eps=0.1_alpha=0.1_nsteps=4
- model_id: 1
  model_name: fgs

In [4]:
# load data
if cfg['dataset'] == 'Ford_A':
    X_train, X_test, y_train, y_test = load_Ford_A()
else:
    X_train, y_train, X_test, y_test = load_UCR(cfg['dataset'])

X_train, X_test, y_train, y_test = transform_data(X_train, X_test, y_train, y_test, slice_data=cfg['slice'])

test_loader = DataLoader(
    MyDataset(X_test, y_test), 
    batch_size=cfg['batch_size'] , 
    shuffle=False
    )

criterion = torch.nn.BCELoss()
n_objects = y_test.shape[0]
device= torch.device(cfg['cuda'] if torch.cuda.is_available() else 'cpu')
attack_func = get_attack(cfg['attack_type'])


model = instantiate(cfg.attack_model).to(device)

disc_model = instantiate(cfg.disc_model).to(device)

if cfg['use_disc_check']:
    disc_model_check = instantiate(cfg.disc_model_check).to(device)
    disc_model_check = load_disc_model(
        copy.deepcopy(disc_model_check),
        model_id=cfg['disc_check_params']['model_id'], 
        path=cfg['disc_path'], 
        model_name=cfg['disc_check_params']['model_name'], 
        device=device
        )
    disc_model_check.eval()
else: 
    disc_model_check = None

alphas = [0]
if 'reg' in cfg['attack_type'] or 'disc' in cfg['attack_type']:
    alphas = cfg['alphas']

for alpha in tqdm(alphas):

    attack_params = dict()

    if 'reg' in cfg['attack_type'] :
        attack_params['alpha'] = alpha

    elif 'disc' in cfg['attack_type']:
        attack_params['alpha'] = alpha
        attack_params['disc_models'] = load_disc_config(
            copy.deepcopy(disc_model),
            cfg['disc_path'], 
            device, 
            cfg['list_reg_model_params']
            )  
        
        attack_params['disc_models'] = [model.eval() for model in  attack_params['disc_models']]
    
    model_path = cfg['model_folder'] + f'model_{cfg["model_id_attack"]}_{cfg["dataset"]}.pth'
    model.load_state_dict(copy.deepcopy(torch.load(model_path)))
    model.eval()

100%|██████████| 1/1 [00:00<00:00, 46.29it/s]


In [9]:
y_true = torch.Tensor()
y_pred = torch.Tensor()

model.eval()
# disc_model = attack_params['disc_models'][0]
# disc_model.train()

for X, y in test_loader:
    pred = model(X.to(device))
    print(pred.flatten()[:10], y.flatten()[:10])

    y_true = torch.cat([y_true, torch.zeros_like(y)])
    y_pred = torch.cat([y_pred, pred.cpu()])
    # if 1 in pred.to(int):
    #     print('1 is detected')
    #     break
        

tensor([0.5119, 0.4826, 0.4832, 0.4910, 0.5135, 0.4938, 0.5077, 0.4878, 0.4871,
        0.4819], device='cuda:1', grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.4898, 0.4949, 0.4938, 0.4998, 0.5002, 0.4833, 0.4893, 0.4822, 0.4902,
        0.4885], device='cuda:1', grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.4895, 0.4787, 0.4848, 0.4896, 0.4830, 0.4989, 0.4949, 0.4826, 0.5187,
        0.4827], device='cuda:1', grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5024, 0.4827, 0.4950, 0.4789, 0.4830, 0.4785, 0.4908, 0.4964, 0.4813,
        0.4824], device='cuda:1', grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.4984, 0.4897, 0.4835, 0.4827, 0.4910, 0.4831, 0.4975, 0.5122, 0.4935,
        0.4981], device='cuda:1', grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.4823, 0.4951, 0.4912, 0.4894, 0.5110, 0.5060,

In [10]:
#accuracy
torch.sum(y_true.flatten() == (y_pred.flatten() > 0.5)) / len(y_true.flatten())

tensor(0.6861)

# check output of the models

In [ ]:
# disc_model_check, attack_params['disc_models']